# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Eduardo Andres Valdes

**ID:** 000340123



**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [39]:
import pandas as pd
import pyarrow
import unicodedata
import re, numpy as np

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.model_selection import train_test_split

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [40]:
# Usar su ID UPB Ejemplo: "000340123"
id_upb = "000340123"

data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1"
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 106485 to 24839
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  string
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), string(1)
memory usage: 1.1 MB


Ejemplo de algunas filas del dataset

In [41]:
data_reviews.sample(5)

,author_review_desc,author_rating
84255,\nAprovechando el éxito en taquilla de la cint...,6
110582,\nNo se lo que me pasa pero estoy llorando la ...,10
99552,"\nEn primer lugar, definiré qué entiendo por ""...",2
134934,"\nBajo mi punto de vista, estamos ante una de ...",10
85794,\nJohnny ha despertado y su sed de sangre es b...,7


Evaluar los valores nulos

In [42]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [43]:
data_reviews.duplicated().sum()

np.int64(4)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [44]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [45]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [47]:
# Analisis exploratorio
X_train.info()

<class 'pandas.core.series.Series'>
Index: 40000 entries, 116556 to 133695
Series name: author_review_desc
Non-Null Count  Dtype 
--------------  ----- 
40000 non-null  string
dtypes: string(1)
memory usage: 625.0 KB


## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [48]:
# Convertir a minúsculas
X_train["texto_min"] = X_train.astype(str).str.lower()

# Eliminar números y caracteres especiales
X_train["texto_limpio"] = X_train["texto_min"].str.replace(
    r"[^a-záéíóúñü\s]", "", regex=True
)

# Eliminar espacios extras
X_train["texto_limpio"] = (
    X_train["texto_limpio"].str.strip().str.replace(r"\s+", " ", regex=True)
)
#Eliminar acentos

def quitar_acentos(text: str) -> str:
    MAP_VOCALES = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u",
    }
    translate = str.maketrans(MAP_VOCALES)
    text = text.translate(translate)
    return text

X_train["texto_sin_acentos"] = X_train["texto_limpio"].apply(quitar_acentos)

# Mostrar resultados
print(X_train["texto_sin_acentos"])


116556    me propuse ver esta cinta sin tener ningun dat...
32252     fantastica pelicula que afortunadamente carece...
109219    la historia al espectador le parece al princip...
103048    adle es una adolescente de años que esta empez...
43746     yo es que me troncho se estrena una pelicula d...
                                ...                        
146914    hay algo en esta pelicula que falla y eso que ...
12540     esto si que es star wars j j abrams ya demostr...
99736     que decir de monty python humor absurdo y dest...
43392     transcurria el año y por aquellos dias christo...
133695    as bestas rodrigo sorogoyen perturbadora human...
Name: author_review_desc, Length: 40000, dtype: object


## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [49]:
# Representación Bag-of-Words con CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

bow_matrix = cv.fit_transform(X_train["texto_sin_acentos"])

print("Dimensiones de la matriz Bag-of-Words:", bow_matrix.shape)
print(
    "Ejemplo de términos (BoW):",
    np.random.choice(cv.get_feature_names_out(), size=10, replace=False),
)


Dimensiones de la matriz Bag-of-Words: (40000, 145190)
Ejemplo de términos (BoW): ['maño' 'monstruosas' 'fuertisima' 'mercantilizado' 'guantes' 'amon'
 'awareness' 'noami' 'dadle' 'liarle']


In [50]:
# Representación TF-IDF con TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train["texto_sin_acentos"])
print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print(
    "Ejemplo de términos (TF-IDF):",
    np.random.choice(tfidf.get_feature_names_out(), size=10, replace=False),
)

Dimensiones de la matriz TF-IDF: (40000, 145190)
Ejemplo de términos (TF-IDF): ['apercibimiento' 'situan' 'cutter' 'allegado' 'stokeriano' 'tomatina'
 'eichiro' 'variaba' 'pildorasdecine' 'inadvertidamente']


## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [51]:
# Naive Bayes#
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# 1. Crear variable binaria de sentimiento
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

# 2. Preparar datos
X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]
# 3. Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42
)
# inicializar el transformador TF-IDF
tfidf = TfidfVectorizer()

# Entrenar y procesar el transformador TF-IDF
# olo con los datos de entrenamiento
x_tfidf_matrix = tfidf.fit_transform(X_train)

nb_classifier = MultinomialNB()
nb_classifier.fit(x_tfidf_matrix, y_train)


,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [53]:
# Usar el transformador TF-IDF entrenado  con lo datos de train
# para procesar los datos de prueba
X_test_processed = tfidf.transform(X_test)

y_pred = nb_classifier.predict(X_test_processed)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))


Accuracy: 0.7396
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.89      0.51      0.65      4708
           1       0.68      0.94      0.79      5292

    accuracy                           0.74     10000
   macro avg       0.79      0.73      0.72     10000
weighted avg       0.78      0.74      0.73     10000



In [54]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.7396
Precision: 0.684564680032958
Recall: 0.9419879062736206
F1 Score: 0.7929059965007158
ROC AUC Score: 0.7270474790501493
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.89      0.51      0.65      4708
           1       0.68      0.94      0.79      5292

    accuracy                           0.74     10000
   macro avg       0.79      0.73      0.72     10000
weighted avg       0.78      0.74      0.73     10000



**Guardar el modelo**

In [55]:
import joblib

## Librerías Usadas

In [ ]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)